<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/Proyecto_TFM/TestingModelos/YoloV8Large_100Epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de detección de espermatozoides usando YoloV8 Large

Se realiza la instalación de las librerías necesarias para el procedimiento:

In [1]:
%%capture
!pip install ultralytics

In [2]:
%%capture
!pip install OpenCV
!pip install Pillow

In [3]:
%%capture
!pip install opencv-contrib-python-headless

Cargamos el DataSet y un video de prueba para realizar la detección:

In [4]:
%%capture
# Carga de DataSet
!wget https://www.dropbox.com/scl/fi/8bjlyvmulkwb4l09lgbzc/dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q&st=7hkm6ksd&dl=0
!wget https://www.dropbox.com/scl/fi/f0wmb3677x28qwt3np0t8/acA1920-155um__22949301__20210628_094625267.avi?rlkey=2lfsr3mspsh5potd64pkb7coc&st=17daydxb&dl=1


In [5]:
%%capture
!mv dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q dataSet.zip
!unzip dataSet.zip

In [6]:
%%capture
!mv acA1920-155um__22949301__20210628_094625267.avi?rlkey=2lfsr3mspsh5potd64pkb7coc test.avi

Cargamos un modelo preentrenado. En este caso usaremos el modelo Large de Yolo para ver el comportamiento en el proceso de entrenamiento y posteriormente aumentar el tamaño del modelo a entrenar.

In [7]:
from ultralytics import YOLO
import cv2 as cv2
from PIL import Image

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8l.pt')

100%|██████████| 83.7M/83.7M [00:00<00:00, 313MB/s]


In [8]:
import os

# Función para corregir las anotaciones
def fix_annotations(annotation_folder):
    for root, dirs, files in os.walk(annotation_folder):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                fix_annotation_file(file_path)

# Función para corregir un archivo de anotación específico
def fix_annotation_file(file_path):
    with open(file_path, "r+") as file:
        lines = file.readlines()
        file.seek(0)
        for line in lines:
            parts = line.strip().split()
            if len(parts) > 5:
                parts = parts[:5]  # Mantener solo las primeras 5 columnas
            fixed_line = " ".join(parts) + "\n"
            file.write(fixed_line)
        file.truncate()

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/val"

# Corregir las anotaciones
fix_annotations(annotation_folder)

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/test"

# Corregir las anotaciones
fix_annotations(annotation_folder)
# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/train"

# Corregir las anotaciones
fix_annotations(annotation_folder)


In [9]:
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data='dataSet.yaml',
                      epochs=100,
                      imgsz=1200,
                      plots = True,
                      batch=-1)

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=dataSet.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=1200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 24.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 107MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[1200] must be multiple of max stride 32, updating to [1216]
AutoBatch: Computing optimal batch size for imgsz=1216
AutoBatch: CUDA:0 (NVIDIA L4) 22.17G total, 0.38G reserved, 0.37G allocated, 21.42G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    43630611       597.1         2.588          37.5         140.5      (1, 3, 1216, 1216)                    list
    43630611        1194         4.675         52.93         93.01      (2, 3, 1216, 1216)                    list
    43630611        2388         8.919         125.9         170.3      (4, 3, 1216, 1216)                    list
    43630611        4777        17.920         288.6           332      (8, 3, 1216, 1216)                    list
CUDA out of memory. Tried to allocate 182.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 70.88 MiB is free. Process 11825 has 22.09 GiB memory in use. Of the allocated memo

...

    100/100      12.3G     0.8015     0.2981      1.035         37       1216: 100%|██████████| 282/282 [02:33<00:00,  1.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:06<00:00,  3.79it/s]

                   all        228       8684      0.865       0.86      0.895      0.633



100 epochs completed in 4.511 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 87.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 87.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:22<00:00,  1.01it/s]


                   all        228       8684      0.886      0.891      0.937       0.66
Speed: 0.4ms preprocess, 19.2ms inference, 0.0ms loss, 17.8ms postprocess per image
Results saved to runs/detect/train


In [10]:
metrics = model.val(split = 'test', plots = True)

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /content/dataSet/labels/test... 177 images, 0 backgrounds, 0 corrupt: 100%|██████████| 177/177 [00:00<00:00, 1238.08it/s]

val: New cache created: /content/dataSet/labels/test.cache



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.82it/s]


                   all        177       2773      0.943      0.916      0.954      0.712
Speed: 0.5ms preprocess, 35.3ms inference, 0.0ms loss, 9.6ms postprocess per image
Results saved to runs/detect/train2


In [11]:
metrics.results_dict

{'metrics/precision(B)': 0.9430001233606587,
 'metrics/recall(B)': 0.9159754778218536,
 'metrics/mAP50(B)': 0.9542042931511807,
 'metrics/mAP50-95(B)': 0.7123746109411659,
 'fitness': 0.7365575791621675}

In [12]:
#model.predict(source = 'dataSet/images/test/image_0.jpg', save = True)
results = model.track(source="test.avi", show=True)

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/test.avi: 768x1216 4 Mov

In [13]:
%%capture
results = model('test.avi', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/test.avi: 768x1216 4 Movils, 20.0ms
video 1/1 (frame 2/60) /content/test.avi: 768x1216 4 Movils, 20.0ms
video 1/1 (frame 3/60) /content/test.avi: 768x1216 4 Movils, 19.9ms
video 1/1 (frame 4/60) /content/test.avi: 768x1216 4 Movils, 19.9ms
video 1/1 (frame 5/60) /content/test.avi: 768x1216 4 Movils, 20.0ms
video 1/1 (frame 6/60) /content/test.avi: 768x1216 4 Movils, 19.9ms
video 1/1 (frame 7/60) /content/test.avi: 768x12

In [14]:
%%capture
results = model('Suppl_1.avi', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/Suppl_1.avi: 768x1216 30 Movils, 20.3ms
video 1/1 (frame 2/60) /content/Suppl_1.avi: 768x1216 30 Movils, 19.8ms
video 1/1 (frame 3/60) /content/Suppl_1.avi: 768x1216 30 Movils, 19.7ms
video 1/1 (frame 4/60) /content/Suppl_1.avi: 768x1216 30 Movils, 19.8ms
video 1/1 (frame 5/60) /content/Suppl_1.avi: 768x1216 30 Movils, 19.7ms
video 1/1 (frame 6/60) /content/Suppl_1.avi: 768x1216 30 Movils, 19.8ms
video 1/1 (frame 7/60) /

In [15]:
# Export the model
model.export()

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 1216, 1216) BCHW and output shape(s) (1, 5, 30324) (83.7 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 8.1s, saved as 'runs/detect/train/weights/best.torchscript' (167.2 MB)

Export complete (11.5s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=1216  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=1216 data=dataSet.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.torchscript'

In [16]:
import shutil
archivo = shutil.make_archive('ModelInformation', 'zip','/content/runs')